## **Handwritten Text Recognition**

---
Handwritten Text Recognition with Tensorflow2 & Keras & IAM Dataset.

Convolutional Recurrent Neural Network. CTC.

Author : Aditya Raj



-






## Dataset used:


Data: [IAM Dataset](http://www.fki.inf.unibe.ch/databases/iam-handwriting-database/download-the-iam-handwriting-database)

Used in this project: [words.tgz](http://www.fki.inf.unibe.ch/DBs/iamDB/data/words/)


In [4]:
!pip install tqdm keras-tqdm

In [5]:
# from requests import Session
# from tqdm import tqdm

# s = Session()
# s.headers.update({'Cookie': 'session=.eJw1yz0OwjAMBtC7eG7aigkycZUPbCUWSYzyQ0GIu1MG9vfeJBmayFO2ZDd76DXOrBnnFwrLc66DJkpo_Y4gO1sYHRc0aQvbVpKBXY_iFNnFvWxVu5bg_uy3LQRhLeR7HTLRaFKVyR9Ph3X9fAEDvS4u.ZwAN8w.KjhKaUXRb0DPwqeZA7Sli08VMdE'})

# response = s.get('https://fki.tic.heia-fr.ch/DBs/iamDB/data/words.tgz', stream=True)

# total_size = int(response.headers.get("content-length", 0))
# block_size = 1024

# with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
#     with open('words.tgz', "wb") as file:
#         for data in response.iter_content(block_size):
#             progress_bar.update(len(data))
#             file.write(data)

# if total_size != 0 and progress_bar.n != total_size:
#     raise RuntimeError("Could not download file")


In [6]:
import os

print(str(round(os.stat('words.tgz').st_size / 1024 / 1024, 2)) + 'Mb')

125.03Mb


In [7]:
import numpy as np
import cv2
import os
import pandas as pd
import string
import matplotlib.pyplot as plt

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [8]:
import tensorflow as tf

#ignore warnings in the output
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [9]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12438531548509485953
xla_global_id: -1
, name: "/device:TPU_SYSTEM:0"
device_type: "TPU_SYSTEM"
memory_limit: 8589934592
locality {
}
incarnation: 12940153592249398075
physical_device_desc: "device: TPU_SYSTEM device"
xla_global_id: -1
, name: "/device:TPU:0"
device_type: "TPU"
memory_limit: 17179869184
locality {
}
incarnation: 10396392055469016097
physical_device_desc: "device: TPU device"
xla_global_id: -1
, name: "/device:TPU:1"
device_type: "TPU"
memory_limit: 17179869184
locality {
}
incarnation: 2965149010298001049
physical_device_desc: "device: TPU device"
xla_global_id: -1
, name: "/device:TPU:2"
device_type: "TPU"
memory_limit: 17179869184
locality {
}
incarnation: 13364761517407516596
physical_device_desc: "device: TPU device"
xla_global_id: -1
, name: "/device:TPU:3"
device_type: "TPU"
memory_limit: 17179869184
locality {
}
incarnation: 7368624470640517564
physical_device_desc: "devi

In [10]:
tf.config.experimental.list_physical_devices('TPU')

[PhysicalDevice(name='/physical_device:TPU:0', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:1', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:2', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:3', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:4', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:5', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:6', device_type='TPU'),
 PhysicalDevice(name='/physical_device:TPU:7', device_type='TPU')]

In [11]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [12]:
strategy = tf.distribute.TPUStrategy(resolver)

In [14]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [15]:
with open('./words.txt') as f:
    contents = f.readlines()

lines = [line.strip() for line in contents]
lines = lines[18:]
lines[0]

'a01-000u-00-00 ok 154 408 768 27 51 AT A'

In [16]:

#  # importing the "tarfile" module
# import tarfile

# # open file
# file = tarfile.open('words.tgz')

# # extracting file
# file.extractall('./words')

# file.close()

In [17]:
max_label_len = 0

char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

# string.ascii_letters + string.digits (Chars & Digits)
# or
# "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

print(char_list, len(char_list))

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, chara in enumerate(txt):
        dig_lst.append(char_list.index(chara))

    return dig_lst

!"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 78


In [18]:
images = []
labels = []

RECORDS_COUNT = 10000

In [19]:
train_images = []
train_labels = []
train_input_length = []
train_label_length = []
train_original_text = []

valid_images = []
valid_labels = []
valid_input_length = []
valid_label_length = []
valid_original_text = []

inputs_length = []
labels_length = []

In [20]:
def process_image(img):
    """
    Converts image to shape (32, 128, 1) & normalize
    """
    w, h = img.shape

    # Aspect Ratio Calculation
    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape

    img = img.astype('float32')

    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)

    img = cv2.subtract(255, img)

    img = np.expand_dims(img, axis=2)

    # Normalize
    img = img / 255

    return img

## Generate train & validation set

In [21]:
for index, line in enumerate(lines):
    splits = line.split(' ')
    status = splits[1]

    if status == 'ok':
        word_id = splits[0]
        word = "".join(splits[8:])

        splits_id = word_id.split('-')
        filepath = 'words/{}/{}-{}/{}.png'.format(splits_id[0],
                                                  splits_id[0],
                                                  splits_id[1],
                                                  word_id)

        # process image
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        try:
            img = process_image(img)
        except:
            continue

        # process label
        try:
            label = encode_to_labels(word)
        except:
            continue

        if index % 10 == 0:
            valid_images.append(img)
            valid_labels.append(label)
            valid_input_length.append(31)
            valid_label_length.append(len(word))
            valid_original_text.append(word)
        else:
            train_images.append(img)
            train_labels.append(label)
            train_input_length.append(31)
            train_label_length.append(len(word))
            train_original_text.append(word)

        if len(word) > max_label_len:
            max_label_len = len(word)

    if index >= RECORDS_COUNT:
        break

In [22]:
train_padded_label = pad_sequences(train_labels,
                             maxlen=max_label_len,
                             padding='post',
                             value=len(char_list))

valid_padded_label = pad_sequences(valid_labels,
                             maxlen=max_label_len,
                             padding='post',
                             value=len(char_list))

In [23]:
train_padded_label.shape, valid_padded_label.shape

((7850, 16), (876, 16))

## Converts to Numpy array

In [24]:
train_images = np.asarray(train_images)
train_input_length = np.asarray(train_input_length)
train_label_length = np.asarray(train_label_length)

valid_images = np.asarray(valid_images)
valid_input_length = np.asarray(valid_input_length)
valid_label_length = np.asarray(valid_label_length)

In [25]:
train_images.shape

(7850, 32, 128, 1)

## Build Model
Convolutional Recurrent Neural Network

In [26]:
def create_model():
  # input with shape of height=32 and width=128
  inputs = Input(shape=(32,128,1))

  # convolution layer with kernel size (3,3)
  conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
  # poolig layer with kernel size (2,2)
  pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

  conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
  pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

  conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)

  conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
  # poolig layer with kernel size (2,1)
  pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

  conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
  # Batch normalization layer
  batch_norm_5 = BatchNormalization()(conv_5)

  conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
  batch_norm_6 = BatchNormalization()(conv_6)
  pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

  conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

  squeezed = Lambda(lambda x: tf.keras.backend.squeeze(x, 1))(conv_7)

  # bidirectional LSTM layers with units=128
  blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(squeezed)
  blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(blstm_1)

  outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

  return tf.keras.Sequential([inputs, conv_1, pool_1, conv_2, pool_2, conv_3, conv_4, pool_4, conv_5, batch_norm_5, conv_6,batch_norm_6, pool_6, conv_7, blstm_1, blstm_2, outputs])
  # model = Model(inputs=[inputs, the_labels, input_length, label_length], outputs=loss_out)
  # return model

  # model to be used at test time
  # act_model = Model(inputs, outputs)

In [28]:
# act_model.summary()

In [31]:
batch_size = 8
epochs = 10
e = str(epochs)
optimizer_name = 'adam'

In [32]:
with strategy.scope():
  # input with shape of height=32 and width=128
  inputs = Input(shape=(32,128,1))

  # convolution layer with kernel size (3,3)
  conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
  # poolig layer with kernel size (2,2)
  pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

  conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
  pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

  conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)

  conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
  # poolig layer with kernel size (2,1)
  pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

  conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
  # Batch normalization layer
  batch_norm_5 = BatchNormalization()(conv_5)

  conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
  batch_norm_6 = BatchNormalization()(conv_6)
  pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

  conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

  squeezed = Lambda(lambda x: tf.keras.backend.squeeze(x, 1))(conv_7)

  # bidirectional LSTM layers with units=128
  blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(squeezed)
  blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(blstm_1)

  outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

  the_labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
  input_length = Input(name='input_length', shape=[1], dtype='int64')
  label_length = Input(name='label_length', shape=[1], dtype='int64')

  def ctc_lambda_func(args):
      y_pred, labels, input_length, label_length = args
      y_pred = y_pred[:, 2:, :]

      return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

  loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, the_labels, input_length, label_length])

  #model to be used at training time
  model = Model(inputs=[inputs, the_labels, input_length, label_length], outputs=loss_out)
  # model = create_model()
  model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = optimizer_name, metrics=['accuracy'])
  filepath="{}o-{}r-{}e-{}t-{}v.hdf5".format(optimizer_name,
                                          str(RECORDS_COUNT),
                                          str(epochs),
                                          str(train_images.shape[0]),
                                          str(valid_images.shape[0]))

  checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
  callbacks_list = [checkpoint]

In [33]:
# batch_size = 8
# epochs = 10
# e = str(epochs)
# optimizer_name = 'adam'

In [34]:
# model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = optimizer_name, metrics=['accuracy'])

# filepath="{}o-{}r-{}e-{}t-{}v.hdf5".format(optimizer_name,
#                                           str(RECORDS_COUNT),
#                                           str(epochs),
#                                           str(train_images.shape[0]),
#                                           str(valid_images.shape[0]))

# checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# callbacks_list = [checkpoint]

In [35]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
history = model.fit(x=[train_images, train_padded_label, train_input_length, train_label_length],
                    y=np.zeros(len(train_images)),
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=([valid_images, valid_padded_label, valid_input_length, valid_label_length], [np.zeros(len(valid_images))]),
                    verbose=1,callbacks=callbacks_list)

Epoch 1/10


## Training Accuracy

In [ ]:
# predict outputs on validation images
prediction = act_model.predict(train_images[150:170])

# use CTC decoder
decoded = K.ctc_decode(prediction,
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]

out = K.get_value(decoded)

# see the results
for i, x in enumerate(out):
    print("original_text =  ", train_original_text[150+i])
    print("predicted text = ", end = '')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    plt.imshow(train_images[150+i].reshape(32,128), cmap=plt.cm.gray)
    plt.show()
    print('\n')

In [ ]:
# plot accuracy and loss
def plotgraph(epochs, acc, val_acc):
    # Plot training & validation accuracy values
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

In [ ]:
plotgraph(epochs, loss, val_loss)

In [ ]:
plotgraph(epochs, acc, val_acc)